## 设置环境

### Python 模块

安装以下Python模块：```bashpip install ipykernel python-dotenv cassio pandas langchain_openai langchain langchain-community langchainhub langchain_experimental openai-multi-tool-use-parallel-patch```

### 加载 `.env` 文件

连接是通过 `cassio` 使用 `auto=True` 参数进行的，并且笔记本使用 OpenAI。您应相应地创建一个 `.env` 文件。对于Cassandra，设置：```bashCASSANDRA_CONTACT_POINTSCASSANDRA_USERNAMECASSANDRA_PASSWORDCASSANDRA_KEYSPACE```对于Astra，设置：```bashASTRA_DB_APPLICATION_TOKENASTRA_DB_DATABASE_IDASTRA_DB_KEYSPACE```例如：```bash# 连接到Astra：ASTRA_DB_DATABASE_ID=a1b2c3d4-...ASTRA_DB_APPLICATION_TOKEN=AstraCS:...ASTRA_DB_KEYSPACE=notebooks# 也设置OPENAI_API_KEY=sk-....```（您还可以修改下面的代码，直接连接到 `cassio`。）

In [ ]:
# 导入 load_dotenv 函数from dotenv import load_dotenv# 调用 load_dotenv 函数，并设置 override 参数为 Trueload_dotenv(override=True)

### 连接到Cassandra

In [ ]:
import os  # 导入os模块，用于操作系统相关的功能import cassio  # 导入cassio模块cassio.init(auto=True)  # 初始化cassio，auto参数设置为Truesession = cassio.config.resolve_session()  # 获取cassio的配置并解析会话if not session:  # 如果会话不存在    raise Exception(        "Check environment configuration or manually configure cassio connection parameters"    )  # 抛出异常，提示检查环境配置或手动配置cassio连接参数keyspace = os.environ.get(  # 获取环境变量ASTRA_DB_KEYSPACE的值，如果不存在则获取CASSANDRA_KEYSPACE的值    "ASTRA_DB_KEYSPACE", os.environ.get("CASSANDRA_KEYSPACE", None))if not keyspace:  # 如果keyspace不存在    raise ValueError("a KEYSPACE environment variable must be set")  # 抛出值错误异常，提示必须设置KEYSPACE环境变量session.set_keyspace(keyspace)  # 设置会话的keyspace

## 设置数据库

这只需要做一次！

### 下载数据

使用的数据集来自Kaggle，是[环境传感器遥测数据](https://www.kaggle.com/datasets/garystafford/environmental-sensor-data-132k?select=iot_telemetry_data.csv)。下一个单元格将下载并解压数据到一个Pandas数据帧中。下一个单元格是手动下载的说明。这一部分的最终结果是你应该有一个名为`df`的Pandas数据帧变量。

#### 自动下载

In [ ]:
从io库中导入BytesIO类和ZipFile类从pandas库中导入pd模块从requests库中导入requests模块设置datasetURL变量为数据集的URL地址使用requests库的get方法发送GET请求，将响应保存在response变量中如果响应状态码为200，表示请求成功    创建一个ZipFile对象，将响应内容作为参数传入    获取压缩文件中的第一个文件名，并将其保存在csv_file_name变量中否则    打印"Failed to download the file"使用ZipFile对象的open方法打开csv文件，并将其作为参数传入pd.read_csv方法中，将读取的结果保存在df变量中

#### 手动下载

您可以下载`.zip`文件并解压缩其中的`.csv`文件。在下一行中添加注释，并相应地调整此`.csv`文件的路径。

In [ ]:
# 使用pandas库的read_csv函数读取名为iot_telemetry_data.csv的文件df = pd.read_csv("/path/to/iot_telemetry_data.csv")

### 将数据加载到Cassandra中

这部分假设存在一个名为`df`的数据框，下面的单元格验证了它的结构。上面的下载部分创建了这个对象。

In [ ]:
# 确保数据框 df 不为空assert df is not None, "Dataframe 'df' must be set"# 预期的列名expected_columns = [    "ts",    "device",    "co",    "humidity",    "light",    "lpg",    "motion",    "smoke",    "temp",]# 确保数据框 df 包含所有预期的列assert all(    [column in df.columns for column in expected_columns]), "DataFrame does not have the expected columns"

创建和加载表格：

In [ ]:
from datetime import UTC, datetime  # 导入UTC和datetime模块from cassandra.query import BatchStatement  # 从cassandra.query模块导入BatchStatement类# 创建传感器表table_query = """CREATE TABLE IF NOT EXISTS iot_sensors (    device text,    conditions text,    room text,    PRIMARY KEY (device))WITH COMMENT = 'Environmental IoT room sensor metadata.';"""session.execute(table_query)  # 执行创建表的CQL语句pstmt = session.prepare(    """INSERT INTO iot_sensors (device, conditions, room)VALUES (?, ?, ?)""")devices = [    ("00:0f:00:70:91:0a", "stable conditions, cooler and more humid", "room 1"),    ("1c:bf:ce:15:ec:4d", "highly variable temperature and humidity", "room 2"),    ("b8:27:eb:bf:9d:51", "stable conditions, warmer and dryer", "room 3"),]for device, conditions, room in devices:    session.execute(pstmt, (device, conditions, room))  # 执行插入数据的预处理语句print("Sensors inserted successfully.")  # 打印插入数据成功的消息# 创建数据表table_query = """CREATE TABLE IF NOT EXISTS iot_data (    day text,    device text,    ts timestamp,    co double,    humidity double,    light boolean,    lpg double,    motion boolean,    smoke double,    temp double,    PRIMARY KEY ((day, device), ts))WITH COMMENT = 'Data from environmental IoT room sensors. Columns include device identifier, timestamp (ts) of the data collection, carbon monoxide level (co), relative humidity, light presence, LPG concentration, motion detection, smoke concentration, and temperature (temp). Data is partitioned by day and device.';"""session.execute(table_query)  # 执行创建表的CQL语句pstmt = session.prepare(    """INSERT INTO iot_data (day, device, ts, co, humidity, light, lpg, motion, smoke, temp)VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""")# 将数据批量插入数据库def insert_data_batch(name, group):    batch = BatchStatement()    day, device = name    print(f"Inserting batch for day: {day}, device: {device}")    for _, row in group.iterrows():        timestamp = datetime.fromtimestamp(row["ts"], UTC)        batch.add(            pstmt,            (                day,                row["device"],                timestamp,                row["co"],                row["humidity"],                row["light"],                row["lpg"],                row["motion"],                row["smoke"],                row["temp"],            ),        )    session.execute(batch)  # 执行批量插入数据的操作# 将数据框中的列转换为适当的类型df["light"] = df["light"] == "true"df["motion"] = df["motion"] == "true"df["ts"] = df["ts"].astype(float)df["day"] = df["ts"].apply(    lambda x: datetime.fromtimestamp(x, UTC).strftime("%Y-%m-%d"))grouped_df = df.groupby(["day", "device"])for name, group in grouped_df:    insert_data_batch(name, group)  # 调用批量插入数据的函数print("Data load complete")  # 打印数据加载完成的消息

In [ ]:
# 打印当前会话的keyspaceprint(session.keyspace)

## 载入工具

演示的Python `import`语句：

In [ ]:
# 导入所需的模块和类from langchain.agents import AgentExecutor, create_openai_tools_agentfrom langchain_community.agent_toolkits.cassandra_database.toolkit import (    CassandraDatabaseToolkit,)from langchain_community.tools.cassandra_database.prompt import QUERY_PATH_PROMPTfrom langchain_community.tools.cassandra_database.tool import (    GetSchemaCassandraDatabaseTool,    GetTableDataCassandraDatabaseTool,    QueryCassandraDatabaseTool,)from langchain_community.utilities.cassandra_database import CassandraDatabasefrom langchain_openai import ChatOpenAI# 创建一个Cassandra数据库工具包对象cassandra_toolkit = CassandraDatabaseToolkit()# 创建一个Cassandra数据库对象cassandra_db = CassandraDatabase()# 创建一个获取Cassandra数据库模式的工具对象get_schema_tool = GetSchemaCassandraDatabaseTool(cassandra_db)# 创建一个获取Cassandra数据库表数据的工具对象get_table_data_tool = GetTableDataCassandraDatabaseTool(cassandra_db)# 创建一个查询Cassandra数据库的工具对象query_tool = QueryCassandraDatabaseTool(cassandra_db)# 创建一个ChatOpenAI对象chat_agent = ChatOpenAI()# 创建一个AgentExecutor对象agent_executor = AgentExecutor()# 创建一个OpenAI工具代理对象openai_agent = create_openai_tools_agent()# 将Cassandra数据库工具包对象添加到代理执行器中agent_executor.add_toolkit(cassandra_toolkit)# 将获取Cassandra数据库模式的工具对象添加到代理执行器中agent_executor.add_tool(get_schema_tool)# 将获取Cassandra数据库表数据的工具对象添加到代理执行器中agent_executor.add_tool(get_table_data_tool)# 将查询Cassandra数据库的工具对象添加到代理执行器中agent_executor.add_tool(query_tool)# 将ChatOpenAI对象添加到代理执行器中agent_executor.add_agent(chat_agent)# 将OpenAI工具代理对象添加到代理执行器中agent_executor.add_agent(openai_agent)

`CassandraDatabase`对象是从`cassio`加载的，尽管它也接受一个`Session`类型的参数作为替代。

In [ ]:
# 创建一个CassandraDatabase实例db = CassandraDatabase(include_tables=["iot_sensors", "iot_data"])# 创建Cassandra数据库工具query_tool = QueryCassandraDatabaseTool(db=db)  # 查询数据库工具schema_tool = GetSchemaCassandraDatabaseTool(db=db)  # 获取数据库模式工具select_data_tool = GetTableDataCassandraDatabaseTool(db=db)  # 获取表数据工具

这些工具可以直接调用：

In [ ]:
# 测试工具print("执行CQL查询:")query = "SELECT * FROM iot_sensors LIMIT 5;"  # 定义查询语句result = query_tool.run({"query": query})  # 使用查询工具执行查询print(result)  # 打印查询结果print("\n获取keyspace的模式:")schema = schema_tool.run({"keyspace": keyspace})  # 使用模式工具获取keyspace的模式print(schema)  # 打印模式print("\n从表中获取数据:")table = "iot_data"  # 定义表名predicate = "day = '2020-07-14' and device = 'b8:27:eb:bf:9d:51'"  # 定义查询条件data = select_data_tool.run(    {"keyspace": keyspace, "table": table, "predicate": predicate, "limit": 5})  # 使用数据选择工具获取数据print(data)  # 打印数据

## 代理配置

In [ ]:
# 导入所需的模块from langchain.agents import Toolfrom langchain_experimental.utilities import PythonREPL# 创建一个 PythonREPL 实例python_repl = PythonREPL()# 创建一个名为 python_repl 的工具实例，用于执行 Python 命令repl_tool = Tool(    name="python_repl",    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",    func=python_repl.run,)

In [ ]:
from langchain import hub# 创建一个ChatOpenAI对象，设置温度为0，模型为"gpt-4-1106-preview"llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")toolkit = CassandraDatabaseToolkit(db=db)# context = toolkit.get_context()# tools = toolkit.get_tools()# 创建一个包含schema_tool、select_data_tool和repl_tool的工具列表tools = [schema_tool, select_data_tool, repl_tool]input = (    QUERY_PATH_PROMPT    + f"""这是您的任务：在 {keyspace} keyspace 中，查找每个设备温度超过23度的总次数，时间为2020年7月14日。 创建一个包含房间名称的摘要报告。如有需要，请使用Pandas。""")prompt = hub.pull("hwchase17/openai-tools-agent")# messages = [#     HumanMessagePromptTemplate.from_template(input),#     AIMessage(content=QUERY_PATH_PROMPT),#     MessagesPlaceholder(variable_name="agent_scratchpad"),# ]# prompt = ChatPromptTemplate.from_messages(messages)# print(prompt)# 选择将驱动代理的LLM# 只有某些模型支持此功能llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)# 构建OpenAI Tools代理agent = create_openai_tools_agent(llm, tools, prompt)print("可用工具:")for tool in tools:    print("\t" + tool.name + " - " + tool.description + " - " + str(tool))

In [ ]:
# 创建一个AgentExecutor对象，传入agent和tools参数，并设置verbose为Trueagent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)# 调用AgentExecutor对象的invoke方法，传入一个包含"input"键的字典作为参数，并将返回结果存储在response中response = agent_executor.invoke({"input": input})# 打印response字典中"output"键对应的值print(response["output"])